In [ ]:
! pip install densratio

In [1]:
# TODO

# 1.
# Make non-linear in such a way that you cannot extrapolate to unseen P(Y,X)

# 2. 
# Use importance estimation to extrapolate.

# 3. 
# Show that when H|Z vs Z|H, extrapolation fails even with importance estimation. 

# 4
# Make true model such that excluding variables should recover a model that is "robust" (P(X|H))

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%load_ext rpy2.ipython

In [3]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 3]

In [4]:
def plot_dat(vars_):
    f, axes = plt.subplots(len(vars_), 1, sharex=True, figsize=(20, 10))

    for (H,title),ax in zip(vars_, axes):
        for i,h in zip(['A', 'B', 'C', 'D'], H):
            sns.distplot(h, label = i, ax=ax)
        ax.legend()
        ax.set_title(title)
        
    # plt.title(title)
    # plt.show()

In [5]:
import numpy as np
from scipy.stats import gamma
import seaborn as sns
from copy import deepcopy

# Y := f(H, W, X, Z, N_Y)
# Y := f(W, X, Z, N_Y)
def fn(h, v, z, w):
    val = -1*(h > np.mean(h))*w*h**2 + w*h + w*z + np.random.normal(0, 5, size = h.shape[0])
    return val/10


def generate_data(N, fn, hidden_cause = True, plot=False, hiddens = [(20,2)]*4, v_conds = [(250,5,5)]*4):
    # H is latent variable, distribution changes (not )
    H = [gamma.rvs(a, loc=b, scale=1, size=N) for a,b in hiddens]

    # V := f(H, N_X)
    V = [(1/(h))*c + np.random.gamma(a, b, size=N) for h,(c,a,b) in zip(H, v_conds)]


    if not hidden_cause:
        V,H = deepcopy(H), deepcopy(V)

    # Z = [gamma.rvs(int(np.random.normal(40, 10)), loc=0, scale=1, size=N) for h in H]
    # Z := f(N_Z) 
    Z = [gamma.rvs(2, loc=1, scale=3, size=N) for h in H]

    # W := f(N_W) -- TREATMENT
    W = [np.random.binomial(1, 0.5, size=N) for h in H]

    # Y:= fn(H, V, Z, W, N_Y)
    Y = [fn(H[idx], V[idx], Z[idx], W[idx]) for idx in range(4)]

    taus = [fn(h,v,z,1) - fn(h,v,z,0) for h,v,z in zip(H,V,Z)]
    
    if plot:
        plot_dat([(H,'H'), (V, 'V') , (Z, 'Z'), (Y, 'Y'), (taus, 'tau')])

    return [(y, np.array([w,v,z]).T, tau) for y,v,z,w,h,tau in zip(Y, V, Z, W, H, taus)]

In [6]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neighbors.kde import KernelDensity
from scipy.stats import gaussian_kde
from scipy.stats import wasserstein_distance
from itertools import combinations

def kde_score(X):
    return gaussian_kde(X).evaluate

def score_wasserstein(dat, phi, model, train_idx):
    y_train, X_train, _ = dat[train_idx]
    model.fit(phi(X_train), y_train)
    resids = [model.predict(phi(x)) - y for y,x,_ in dat]
    dists = [wasserstein_distance(resids[0], r) for r in resids]
    mss = [np.mean(r**2) for r in resids]
    score = np.sum(dists)
    return score


def filter_dat(d, idxs):
    return [(y, x[:,idxs],tau) for y,x,tau in d]

def search_wasserstein(dat, phi, model, train_idx):
    s = dat[0][1].shape[1]

    combs = [j for i in range(1,s) 
             for j in combinations(range(1, s), i)]

    combs = [[0] + list(c) for c in combs]

    scores = [score_wasserstein(filter_dat(dat, i), phi, model, train_idx) for i in combs]

    return combs[np.argmin(scores)]


def run_model(dat, model, phi, train_idx, target_idx, use_weights = None, model_search = False):
    X_train, X_target = dat[train_idx][1], dat[target_idx][1]

    if model_search:
        # exclude the target in the model search
        dd = [d for i,d in enumerate(dat) if i != target_idx]
        idxs = search_wasserstein(dd, phi, model, train_idx)
        # print('Choosing variables: ', idxs)
    else:
        idxs = range(0, X_train.shape[1])


    if use_weights is not None:
        ps, pt = kde_score(X_train[:, use_weights]), kde_score(X_target[:, use_weights])
        weights = pt(X_train[:, use_weights]) / ps(X_train[:, use_weights])
    else:
        weights = np.ones(X_train.shape[0])


    d = [(phi(x[:, idxs]), x[:, 0], y) for y,x,tau in dat]

    model.fit(d[train_idx][0], d[train_idx][2], sample_weight=weights)

    y0 = [np.mean(model.predict(p[w == 0])) for p,w,y in d]
    y1 = [np.mean(model.predict(p[w == 1])) for p,w,y in d]

    pred_ates = [a-b for a,b in zip(y1, y0)]
    _, _, true_ates = zip(*dat)

    return [np.round(np.abs(p-np.mean(t)), 4) for p,t in zip(pred_ates, true_ates)]

phi = PolynomialFeatures(degree=2, include_bias=False).fit_transform

In [7]:
hiddens = [(25, 2), (4, 15), (4, 20), (2, 25)]
# hiddens = [(10,2)]*4
v_conds = [(500,2,2)]*4
# v_conds = [(250,2,30), (250,5,10), (500,15,5), (500,40,2)]

dat = generate_data(2000,
                    fn, 
                    hidden_cause = False, 
                    plot = False, 
                    hiddens = hiddens,
                    v_conds = v_conds)

y, X, tau = dat[0]

In [8]:
from trees import TransferTreeRegressor, build_tree, Node
from criteria import mse, transfer, causal_tree_criterion

In [9]:
weights = np.random.exponential(scale = 4.0, size = y.shape[0])

weights = np.ones(y.shape[0])

In [10]:
ys, Xs, taus = zip(*dat)

ys_source, Xs_source = np.concatenate(ys[:-1]), np.concatenate(Xs[:-1])
ys_target, Xs_target = ys[-1], Xs[-1]

phi_source = phi(Xs_source[:, 1:])
phi_target = phi(Xs_target[:, 1:])

treatment = Xs_source[:, 0]
N = ys[0].shape[0]
context_idxs = np.array([j for i,_ in enumerate(ys[:-1]) for j in [i]*N])

In [ ]:
model = TransferTreeRegressor(criterion = transfer, max_depth = 5, min_samples_leaf = 10)

model.fit(phi_source, ys_source, treatment=treatment, context_idxs=context_idxs, target_X=phi_target)

In [ ]:
preds = [model.predict(phi(x[:, 1:])) for x in Xs]
[np.mean((p-t)**2) for p,t in zip(preds, taus)]

In [ ]:
np.mean((np.mean(taus[:-1]) - taus[-1])**2)

In [ ]:
[np.mean((np.mean(t) - t)**2) for t in taus]

# Causal Tree Test

In [13]:
import pandas as pd


def get_simulation(fi = 'simulation-1-athey.csv'):
    df = pd.read_csv(fi)
    X = df.iloc[:, :4].values
    y = df.y.values
    treatment = df.treatment.values
    return X, y, treatment

sim_X, sim_y, sim_treatment = get_simulation()

In [14]:
def eta(X):
    return 0.5 * X[:, 0] + X[:, 1]

def kappa(X):
    return 0.5 * X[:, 0]

def gen_y(X, w):
    return eta(X) + 0.5 * (2*w - 1) * kappa(X) + np.random.normal(0, 0.01)


X = np.random.normal(0, 1, (1000, 10))
w = np.random.binomial(1, 0.5, 1000)
y = gen_y(X, w)

In [81]:
def eta(X):
    return 0.5 * np.sum(X[:, :2], axis=1) + np.sum(X[:, 2:], axis=1)

def kappa(X):
    return np.sum(X[:, :2] * (X[:, :2] > 0).astype(int), axis=1)

def gen_y(X, w):
    return eta(X) + 0.5 * (2*w - 1) * kappa(X) + np.random.normal(0, 0.01, size = X.shape[0])


N = 2000
S = int(N/2)
X = np.random.normal(0, 1, (N, 6))
treatment = np.random.binomial(1, 0.5, N)
weights = np.ones(N)
y = gen_y(X, treatment)

In [60]:
sim_dat = pd.concat([pd.DataFrame(X, columns = range(1, 7)), 
                     pd.DataFrame({ 'y': y, 'treatment': treatment, 'tau': kappa(X)})], axis=1)

sim_dat.to_csv('sim_dat-2.csv', index=False)

In [108]:
from sklearn.model_selection import ParameterGrid


def make_nested_grid(params):
    return ParameterGrid({ key: list(ParameterGrid(value))
                           for key, value in params.items()})
        

def model_search(model, X, y, treatment, weights, init_params, fit_params, splitter):
    grid = make_nested_grid({'fit_params': fit_params, 
                             'init_params': init_params})

    all_scores = []

    for g in grid:
        model.set_params(**g['init_params'])

        scores = []

        for train_idx, test_idx in splitter.split(X, treatment):
            model.fit(X[train_idx, :], 
                      y[train_idx], 
                      treatment = treatment[train_idx], 
                      sample_weight = weights[train_idx],
                      min_samples = 25, 
                      **g['fit_params'])

            score = model.score(X[test_idx, :],
                                y[test_idx], 
                                treatment = treatment[test_idx], 
                                sample_weight = weights[test_idx],
                                min_samples = 1,
                                **g['fit_params'])
            
            
            scores.append(score)

        all_scores.append((g, np.mean(scores)))
       
    return all_scores        

In [107]:
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits = 3)

model = TransferTreeRegressor(criterion = causal_tree_criterion, 
                              min_samples_leaf = 4,
                              alpha = 0.1)


fit_params = {'var_weight': [0.0, 0.5] }
init_params = {'alpha': [0.05, 0.2 ] }

model_search(model, X, y, treatment, weights, init_params, fit_params, cv)

[({'fit_params': {'var_weight': 0.0}, 'init_params': {'alpha': 0.05}},
  [-0.9256971598389312, -0.9340066412021991, -1.3319259692980923],
  -1.0638765901130742),
 ({'fit_params': {'var_weight': 0.0}, 'init_params': {'alpha': 0.2}},
  [-0.4806708403836387, -0.8542773366523394, -0.7083604664739335],
  -0.6811028811699705),
 ({'fit_params': {'var_weight': 0.5}, 'init_params': {'alpha': 0.05}},
  [-0.14121154220346985, -0.35031742665049326, -0.4497601253908491],
  -0.3137630314149374),
 ({'fit_params': {'var_weight': 0.5}, 'init_params': {'alpha': 0.2}},
  [-0.18740142970628498, -0.37803607490958757, -0.324448676144225],
  -0.29662872692003256)]

In [142]:
from sklearn.metrics import r2_score

model = TransferTreeRegressor(criterion = causal_tree_criterion, 
                              min_samples_leaf = 4,
                              alpha = 0.05)

model.fit(X[:S], y[:S], treatment=treatment[:S], min_samples=25, var_weight = 0.5)

print(r2_score(kappa(X[S:]), model.predict(X[S:])))
model.tree

0.18348292262212196


|--dim: 1, thresh: 1.7813, score: 0.6812, gain: 0.1134, tot_gain: 0.4526 
   |--dim: 0, thresh: 1.5963, score: 0.5670, gain: 0.0983, tot_gain: 0.3392 
      |--dim: 2, thresh: 1.0462, score: 0.4396, gain: 0.1301, tot_gain: 0.2408 
         |--dim: 5, thresh: -0.8198, score: 0.7156, gain: 0.0488, tot_gain: 0.1108 
            |--pred: 0.7875, score: -0.0435, se: 0.2324, N: 170 
            |--dim: 1, thresh: 0.4737, score: 0.8209, gain: 0.0620, tot_gain: 0.0620 
               |--pred: 0.4929, score: -0.0374, se: 0.1743, N: 410 
               |--pred: 1.5716, score: -0.2101, se: 0.2719, N: 181 
         |--pred: -0.7216, score: -0.0198, se: 0.3356, N: 134 
      |--pred: 2.5692, score: -0.1636, se: 0.5209, N: 54 
   |--pred: 2.8571, score: -0.1925, se: 0.5541, N: 51 

In [145]:
preds = pd.DataFrame(model.predict(X[:S], interval = True), columns = ['pred', 'se'])

preds.assign(true = kappa(X[:S])) \
     .pipe(lambda df: df.assign(within_int = (df.pred + 2*df.se >= df.true) & (df.pred - 2*df.se <= df.true))) \
     .pipe(lambda df: df['within_int'].sum() / df.shape[0])


0.411

In [405]:
osplit = sim_dat.iloc[:, 4] > 0.1237

sim_dat.y[(split) & (sim_dat.treatment == 1)].mean() - sim_dat.y[(split) & (sim_dat.treatment == 0)].mean()

0.7078416678365076

In [406]:
sim_dat.y[(~split) & (sim_dat.treatment == 1)].mean() - sim_dat.y[(~split) & (sim_dat.treatment == 0)].mean()

0.8734375006556282

In [417]:
split = sim_dat.iloc[:, 0] > 0

# split = sim_dat.iloc[:, 4] > 0.1237

right = sim_dat.y[(split) & (sim_dat.treatment == 1)].mean() - sim_dat.y[(split) & (sim_dat.treatment == 0)].mean()

left = sim_dat.y[(~split) & (sim_dat.treatment == 1)].mean() - sim_dat.y[(~split) & (sim_dat.treatment == 0)].mean()

np.abs(left - right)

0.925117681406297

In [398]:
sim_dat.y[(sim_dat.iloc[:, 4] < 0.1237)]

3      -0.863831
5      -1.300832
6       0.716965
9      -5.354481
10     -1.920049
          ...   
9994   -1.482248
9995   -3.187449
9996   -1.511569
9997   -3.340039
9998   -1.317045
Name: y, Length: 5415, dtype: float64

In [383]:
model.tree

|--dim: 4, thresh: 0.1237, gain: 1.3143 
   |--dim: 3, thresh: -0.1165, gain: 0.6556 
      |--dim: 2, thresh: 0.0963, gain: 0.2884 
         |--pred: 0.9733, score: 0.6167, N: 688 
         |--pred: 0.9348, score: 0.4541, N: 562 
      |--dim: 5, thresh: -0.1778, gain: 0.3650 
         |--pred: 0.6768, score: 0.4965, N: 588 
         |--pred: 0.8700, score: 0.8325, N: 877 
   |--dim: 5, thresh: -0.0986, gain: 0.6409 
      |--dim: 2, thresh: -0.5321, gain: 0.3041 
         |--pred: 0.6900, score: 0.2758, N: 302 
         |--pred: 0.6315, score: 0.7269, N: 788 
      |--dim: 2, thresh: 0.0480, gain: 0.3803 
         |--pred: 0.9040, score: 0.6026, N: 635 
         |--pred: 0.8231, score: 0.5235, N: 560 

In [200]:
np.mean(model.predict(X[500:]))

-0.09800523292342495

In [201]:
np.mean(sim_dat.y[sim_dat.treatment == 1]) - np.mean(sim_dat.y[sim_dat.treatment == 0])

-0.07805119891851248

In [59]:
model = TransferTreeRegressor(criterion = mse, max_depth = 5, min_samples_leaf = 10)

model.fit(phi(X), y, sample_weight = weights)
preds = model.predict(phi(X))
# np.mean((preds - y)**2)

np.histogram(preds)

(array([  15,    0,   41,   23,   95,   47,   87,  103,  406, 1183]),
 array([-129.40835129, -116.21549089, -103.02263049,  -89.82977009,
         -76.63690969,  -63.44404929,  -50.25118889,  -37.05832849,
         -23.86546809,  -10.67260769,    2.52025272]))

In [60]:
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor(max_depth=5, min_samples_leaf=10)

model.fit(phi(X), y, sample_weight = weights)
preds = model.predict(phi(X))
np.mean((preds - y)**2)

(array([  15,    0,   41,   23,   95,   47,   87,  103,  406, 1183]),
 array([-129.40835129, -116.21549089, -103.02263049,  -89.82977009,
         -76.63690969,  -63.44404929,  -50.25118889,  -37.05832849,
         -23.86546809,  -10.67260769,    2.52025272]))

In [34]:
model = DecisionTreeRegressor(max_depth=4, min_samples_leaf=20)

np.mean(np.array([run_model(dat, model, phi, 0, 3, use_weights = None, model_search = True) for i in range(10)]), 0)

array([ 0.3661, 20.3762,  0.4404,  7.6768])

In [35]:
np.mean(np.array([run_model(dat, model, phi, 0, 3, use_weights = None, model_search = False) for i in range(10)]), 0)

array([ 0.3656, 41.3072, 15.5   ,  1.5563])

In [36]:
np.mean(np.array([run_model(dat, model, phi, 0, 3, use_weights = 1, model_search = True) for i in range(10)]), 0)

array([11.7473, 32.487 , 11.613 ,  4.5725])

In [37]:
np.mean(np.array([run_model(dat, model, phi, 0, 3, use_weights = 1, model_search = False) for i in range(10)]), 0)

array([12.42759, 25.75614,  5.67631,  3.50874])

In [38]:
model = TransferTreeRegressor(criterion=mse, max_depth=4, min_samples_leaf=20)

np.mean(np.array([run_model(dat, model, phi, 0, 3, use_weights = None, model_search = True) for i in range(10)]), 0)

array([ 0.3661, 20.3762,  0.4404,  7.6768])

In [39]:
np.mean(np.array([run_model(dat, model, phi, 0, 3, use_weights = None, model_search = False) for i in range(10)]), 0)

array([ 0.3656, 41.3072, 15.5   ,  1.5563])

In [52]:
np.mean(np.array([run_model(dat, model, phi, 0, 3, use_weights = 1, model_search = True) for i in range(10)]), 0)

array([11.3818, 32.1958, 11.2033,  3.5604])

In [51]:
np.mean(np.array([run_model(dat, model, phi, 0, 3, use_weights = 1, model_search = False) for i in range(10)]), 0)

array([10.9098,  4.9235,  3.2745,  3.7268])

In [42]:
model = LinearRegression(fit_intercept=False)
run_model(dat, model, phi, 0, 1, use_weights = None, model_search = False)

[0.3605, 21.2762, 16.201, 7.1304]

In [31]:
run_model(dat, model, phi, 0, 3, use_weights = None, model_search = True)

[0.4141, 20.8479, 0.2354, 6.2956]

In [32]:
run_model(dat, model, phi, 0, 1, use_weights = 1, model_search = False)

[12.3876, 41.752, 16.8672, 4.4122]

In [33]:
run_model(dat, model, phi, 0, 3, use_weights = 1, model_search = True)

[11.362, 31.8239, 11.2857, 4.7136]

In [1802]:
# get residuals for "sets" separately
# compute distance between residuals
# optimize squared errors + penalty for residual distance

# search for "sets" by looking at residuals and fitting a mixture model
# then optimize to remove that mixture...

# set up an adversarial problem: the adversary tries to find a 
# mixture model in your reiduals, the classifier tries to make force the
# adversary to fit a 1-component mixture, for example... 